In [ ]:
#References
# https://huggingface.co/blog/fine-tune-xlsr-wav2vec2

# Install Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install soundfile
!pip install jiwer
# !pip install tqdm==4.43.0

     |████████████████████████████████| 186 kB 4.9 MB/s 
     |████████████████████████████████| 243 kB 10.0 MB/s 
     |████████████████████████████████| 118 kB 8.9 MB/s 
     |████████████████████████████████| 2.1 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 39.6 MB/s 
     |████████████████████████████████| 895 kB 49.1 MB/s 
     |████████████████████████████████| 50 kB 2.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149860 sha256=11ef0c1df653c1ee99e16d467f9ca139f35219a01d74cf95f2c13c5a0fede013
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


# Prepare Data

**Split into train and test two files**

In [ ]:
import csv
import os 

dataset = []

path = "/content/drive/MyDrive/KunFYP/text_dataset_improved.csv"

c = 0
with open(path) as csv_file:
  csv_reader = csv.reader(csv_file)
  for row in csv_reader:
    if c == 0:
      c = 1
    else:
      data = []
      s = row[0]
      s2 = row[1]
      data.append(s)
      data.append(s2)
      dataset.append(data)

len_train = int(len(dataset)*0.3)
len_test = int(len(dataset)*0.06)

#write to train and test files

train_path = "/content/drive/MyDrive/KunFYP/text_dataset_train.csv"
test_path = "/content/drive/MyDrive/KunFYP/text_dataset_test.csv"

with open(train_path,'w',newline='') as file:
      writer = csv.writer(file)
      writer.writerow(["Filename", "Text"])
      for x in range(len_train):
        writer.writerow([dataset[x][0],dataset[x][1]])

with open(test_path,'w',newline='') as file:
      writer = csv.writer(file)
      writer.writerow(["Filename", "Text"])
      for x in range(len(dataset) - len_test, len(dataset)):
        writer.writerow([dataset[x][0],dataset[x][1]])

In [ ]:
from datasets import load_dataset, load_metric

path = '/content/drive/MyDrive/KunFYP/audio_dataset'
text_path = '/content/drive/MyDrive/KunFYP/text_dataset_improved.csv'

dataset_train = load_dataset("csv", data_files = [train_path])
dataset_test = load_dataset("csv", data_files = [test_path])

print(dataset_train)
print(dataset_test)


Using custom data configuration default-b1366354a2a3e860


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b1366354a2a3e860/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.


Using custom data configuration default-e36e8899f235dadb


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e36e8899f235dadb/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['Filename', 'Text'],
        num_rows: 2420
    })
})
DatasetDict({
    train: Dataset({
        features: ['Filename', 'Text'],
        num_rows: 484
    })
})


**Check set values**

In [ ]:
print(dataset_train["train"][0])
print(dataset_test["train"][0])

{'Filename': 'A2.wav', 'Text': 'وہ روز صبح سکول جاتے'}
{'Filename': 'C1871.wav', 'Text': 'پہلی مکمل'}


**Display sample data**

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
    print(f'length of dataset: {len(dataset)}')
show_random_elements(dataset_train["train"].remove_columns(["Filename"]),10)

,Text
0,میرکے معاصرین میں سودا نے بھی مرثیے کہے
1,میر کو مقام دیا کہ وہ
2,جو
3,اردو کو
4,کھلیان
5,کس نوعیت کے اضافی
6,اسے دیکھ کر
7,فارسی سندری
8,اس اتوار کو تلاش
9,اردو زبان کا


length of dataset: 2420


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["Text"] = re.sub(chars_to_ignore_regex, '', batch["Text"]).lower()
    return batch

dataset_train = dataset_train.map(remove_special_characters)
dataset_test = dataset_test.map(remove_special_characters)

In [ ]:
show_random_elements(dataset_train["train"].remove_columns(["Filename"]))


,Text
0,اسے 12 بیرونی حملہ آوروں نے عقل
1,کہ معاشرے میں قنوطیت اور مایوسی کے
2,کردار نگاری کے حوالے سے
3,ان کا تصور غم
4,آوازوں سے
5,آئندہ الیکشن
6,ہم
7,شعرائے کرام واردات قلبی کا اظہار کر دیتے
8,اور سونم پنجاب میں اردو
9,اب یہ سوال


length of dataset: 2420


# Extract Characters

In [ ]:
# def extract_all_chars(batch):
#   all_text = " ".join(batch["Text"])
#   vocab = list(set(all_text))
#   return {"vocab": [vocab], "all_text": [all_text]}

# vocabs = dataset_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset_train.column_names["train"])
# vocabs2 = dataset_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset_test.column_names["train"])
# print(vocabs['train']['vocab'])


[['4', 's', 'د', 'c', 'ق', 'ش', 'ذ', 'خ', 'ء', ' ', 'و', 'چ', 'ن', 'ر', '0', 'ٰ', '1', 'e', 'b', '5', 'ز', 'غ', 'k', 'ہ', 'ض', 'ک', 'ظ', 'ل', 'ا', 'a', '3', 'ج', 'ع', 'ۃ', 'x', 'ت', 'ئ', 'ڈ', 'ی', '7', 'ف', 'ث', '9', 'ے', 'س', 'ٹ', 'ط', 'َ', 'ؤ', 'ب', 'م', 'ں', 'گ', 'l', '2', 'آ', 'ھ', '\u200f', 'ڑ', 'پ', 'ح', 'ص']]


In [ ]:
urdu_words = ['د', 'ق', 'ش', 'ذ', 'خ', 'ء', ' ', 'و', 'چ', 'ن', 'ر', 'ز', 'غ', 'ہ', 'ض', 'ک', 'ظ', 'ل', 'ا', 'ج', 'ع', 'ۃ', 'ت', 'ئ', 'ڈ', 'ی', 'ف', 'ث', 'ے', 'س', 'ٹ', 'ط', 'ؤ', 'ب', 'م', 'ں', 'گ', 'آ', 'ھ', 'ڑ', 'پ', 'ح', 'ص']

#convert to a dictionary
vocab_dict = {v:k for k,v in enumerate(urdu_words)}
vocab_dict

{' ': 6,
 'ء': 5,
 'آ': 37,
 'ؤ': 32,
 'ئ': 23,
 'ا': 18,
 'ب': 33,
 'ت': 22,
 'ث': 27,
 'ج': 19,
 'ح': 41,
 'خ': 4,
 'د': 0,
 'ذ': 3,
 'ر': 10,
 'ز': 11,
 'س': 29,
 'ش': 2,
 'ص': 42,
 'ض': 14,
 'ط': 31,
 'ظ': 16,
 'ع': 20,
 'غ': 12,
 'ف': 26,
 'ق': 1,
 'ل': 17,
 'م': 34,
 'ن': 9,
 'و': 7,
 'ٹ': 30,
 'پ': 40,
 'چ': 8,
 'ڈ': 24,
 'ڑ': 39,
 'ک': 15,
 'گ': 36,
 'ں': 35,
 'ھ': 38,
 'ہ': 13,
 'ۃ': 21,
 'ی': 25,
 'ے': 28}

In [ ]:
# vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs2["train"]["vocab"][0]))

# vocab_dict = {v: k for k, v in enumerate(vocab_list)}
# vocab_dict

{' ': 59,
 '0': 4,
 '1': 60,
 '2': 31,
 '3': 15,
 '4': 18,
 '5': 21,
 '6': 51,
 '7': 56,
 '8': 12,
 '9': 25,
 'a': 0,
 'b': 45,
 'c': 10,
 'e': 11,
 'k': 1,
 'l': 26,
 's': 62,
 'x': 16,
 'ء': 32,
 'آ': 27,
 'ؤ': 22,
 'ئ': 40,
 'ا': 42,
 'ب': 33,
 'ت': 28,
 'ث': 13,
 'ج': 41,
 'ح': 63,
 'خ': 57,
 'د': 34,
 'ذ': 58,
 'ر': 44,
 'ز': 9,
 'س': 35,
 'ش': 52,
 'ص': 19,
 'ض': 48,
 'ط': 30,
 'ظ': 50,
 'ع': 24,
 'غ': 54,
 'ف': 36,
 'ق': 49,
 'ل': 6,
 'م': 20,
 'ن': 7,
 'و': 46,
 'َ': 39,
 'ٰ': 3,
 'ٹ': 23,
 'پ': 43,
 'چ': 29,
 'ڈ': 61,
 'ڑ': 38,
 'ک': 17,
 'گ': 37,
 'ں': 5,
 'ھ': 55,
 'ہ': 8,
 'ۃ': 53,
 'ی': 47,
 'ے': 2,
 '\u200f': 14}

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

45


In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

# Create Wav2Vec2 Feature Extractor

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/drive/MyDrive/KunFYP/wav2vec2-processor-kunfyp-urdu")

# Preprocess Data

In [ ]:
print(dataset_train["train"][0])

{'Filename': 'A2.wav', 'Text': 'وہ روز صبح سکول جاتے'}


In [ ]:
import soundfile as sf
import librosa

def speech_file_to_array_fn(batch):
    init_path = "/content/drive/MyDrive/KunFYP/audio_dataset/"
    speech_array, sampling_rate = librosa.load(init_path + batch["Filename"],sr =16000)
    # speech_array, sampling_rate = sf.read(init_path + batch["Filename"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["Text"]
    return batch

dataset_train = dataset_train.map(speech_file_to_array_fn, remove_columns=dataset_train.column_names["train"], num_proc=4)
dataset_test = dataset_test.map(speech_file_to_array_fn, remove_columns=dataset_test.column_names["train"], num_proc=4)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(dataset_train["train"]))

# print(dataset_train["train"][0]["speech"])
ipd.Audio(data=np.asarray(dataset_train["train"][rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(dataset_train["train"]))

print("Target text:", dataset_train["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(dataset_train["train"][rand_int]["speech"]).shape)
print("Sampling rate:", dataset_train["train"][rand_int]["sampling_rate"])

Target text: بات کریں گے
Input array shape: (27296,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

dataset_prepared = dataset_train.map(prepare_dataset, remove_columns=dataset_train.column_names["train"], batch_size=8, num_proc=4, batched=True)
dataset_prepared2 = dataset_test.map(prepare_dataset, remove_columns=dataset_test.column_names["train"], batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


# Training and Evaluation

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/KunFYP/wav2vec2-model-kunfyp-urdu",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_prepared["train"],
    eval_dataset=dataset_prepared2["train"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/KunFYP/wav2vec2-processor-kunfyp-urdu")
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/KunFYP/wav2vec2-model-kunfyp-urdu (1)/checkpoint-800")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
input_dict = processor(dataset_prepared2["train"]["input_values"][0], return_tensors="pt", padding=True)

logits = model(input_dict.input_values).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


# Predictions

In [ ]:
dataset_test_transcription = load_dataset("csv", data_files = [test_path])
print(dataset_test_transcription)

Using custom data configuration default-e36e8899f235dadb
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-e36e8899f235dadb/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)


DatasetDict({
    train: Dataset({
        features: ['Filename', 'Text'],
        num_rows: 484
    })
})


In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(dataset_test_transcription["train"]["Text"][1].lower())

Prediction:
ہر[UNK]کیفییت[UNK]میں[UNK]ربتالا[UNK]کیزات[UNK]موچود[UNK]ہے

Reference:
ادبی کتاب قرار پائی ہے
